In [1]:
import pickle
import json
import urllib
import requests

In [2]:
BASE_PATH = 'E:\Datasets\TWITTER_SOCG\%s'
file_name = 'con_geo.json'
file = BASE_PATH%(file_name)

In [3]:
def get_location_data(location_name):
    '''
    This function gets the location details for a given text location
    https://nominatim.openstreetmap.org/search.php?q=california&format=json&limit=1
    '''
    base_url = 'https://nominatim.openstreetmap.org/search.php?%s'
    search_term = {'format':'json','limit':1}
    search_term['q'] = location_name
    url_args = urllib.parse.urlencode(search_term)
    url = base_url%(url_args)
    response = requests.get(url)
    res = [{}]
    if response.status_code != 200:
        #Some error handling
        print("Unable to fetch data from API, Reponse = ",response.status_code)
    else:
        try:
            res = json.loads(response.content)
        except Exception as e:
            print("Error for content,",e)
    return res[0]

In [4]:
def fix_JSON(json_message=None):
    '''This function fixes errors in the JSON if any and return a json dictionary'''
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        e_message = str(e)
        idx_to_replace = int(e_message.split(' ')[-1].replace(')',''))   
        
        #print(e_message,idx_to_replace)
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [5]:
def data_filter(arg,params):
    res = {}
    for p in params:
        res[p] = arg[p]
    return res

In [6]:
data = {}
locations = {}

In [7]:
criteria = ['id','location','description','followers_count','friends_count'] #user filtering criteria

In [ ]:
## Filtering user data
count = 0
with open(file) as f:
    for line in f:
        line = line[1:-1]
        tmp_all = fix_JSON(line)
        break
        user = tmp_all['user'] 
        data[user['id']] = data_filter(user,criteria)
        location_name = data[user['id']].get('location',None)
        print(user['id'],location_name)
        if location_name is not None and location_name not in locations:
            locations[location_name] = get_location_data(location_name)

In [17]:
count

1

In [14]:
tmp_all.keys()

dict_keys(['475930654', '708399669', '2822905802', '815434441644355584', '825210153658097669', '375220990', '534389949', '2405424169', '1038104010723651586', '142867337', '1028519311', '1452328698', '173722557', '2151272689', '27699053', '310092586', '1660632920', '1283972408', '180414781', '860764812', '2352500341', '952298599', '821009664414482433', '453889330', '2389937450', '2283257755', '1073503328', '4072221977', '388009314', '4698300728', '68195277', '972782022006583296', '264901976', '2581417824', '25837140', '321599111', '56242215', '31182525', '4845714213', '235319495', '1451773710', '15337584', '910823640', '3344107473', '192684183', '939677695662837760', '707515009', '783441819350818816', '59016329', '38180277', '1138377523', '230551751', '60482659', '27947756', '2630575578', '3864037337', '2734782059', '2235271285', '15701453', '756696103102185472', '193985495', '4368957793', '4873940627', '822569799075135490', '49336738', '3310853512', '25150161', '3099596547', '963519025

In [15]:
tmp_all['475930654']

[{'date': 'Fri Nov 30 16:44:49 +0000 2018',
  'place': {'attributes': {},
   'bounding_box': {'coordinates': [[[33.9879342, -3.1894778],
      [38.0789993, -3.1894778],
      [38.0789993, 4.631608],
      [33.9879342, 4.631608]]],
    'type': 'Polygon'},
   'contained_within': [],
   'country': 'Kenya',
   'country_code': 'KE',
   'full_name': 'Rift Valley, Kenya',
   'id': '01065a114b362889',
   'name': 'Rift Valley',
   'place_type': 'admin',
   'url': 'https://api.twitter.com/1.1/geo/id/01065a114b362889.json'}},
 {'date': 'Sun Oct 07 10:02:48 +0000 2018', 'place': None},
 {'date': 'Sat Sep 22 12:04:37 +0000 2018',
  'place': {'attributes': {},
   'bounding_box': {'coordinates': [[[34.5465996, 0.5511975],
      [34.569064, 0.5511975],
      [34.569064, 0.6001177],
      [34.5465996, 0.6001177]]],
    'type': 'Polygon'},
   'contained_within': [],
   'country': 'Kenya',
   'country_code': 'KE',
   'full_name': 'Bungoma, Kenya',
   'id': '00bffa1f7e200483',
   'name': 'Bungoma',
   'pl

In [9]:
print("SIZE of data = ",len(data),len(locations))

SIZE of data =  0 0


In [ ]:
## Getting all location name details
for k,v in data.items():
    if k not in locations:
        location_name = v.get('location',None)
        if location_name is not None:
            locations[location_name] = get_location_data(location_name)

In [ ]:
## Dumping location data
with open('location_by_user.pkl', 'wb') as handle:
    pickle.dump(locations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Generating data for plotting
coord = []
for k,v in locations.items():
    if len(v) != 0:
        coord.append((float(v[0]['lat']),float(v[0]['lon'])))

In [ ]:
## Dumping coordinate data
with open('coord_user_1.pkl', 'wb') as handle:
    pickle.dump(coord, handle, protocol=pickle.HIGHEST_PROTOCOL)